In [1]:
import fastai.datasets
import fastai.vision
import torch
import torch.nn as nn
import torch.nn.functional as F
from callbacks import * 
from misc import *
import core
from functools import partial
import time
import pdb
from models import *
from optim import *

In [2]:
func = lambda x: str(x)[46:].rstrip('.jpg1234567890').rstrip('_')
url = fastai.datasets.URLs.PETS
path = fastai.datasets.untar_data(url)
data = (fastai.vision.ImageList.from_folder(path/'images')
        .split_by_rand_pct()
        .label_from_func(func)
        .transform(fastai.vision.get_transforms(), size=224)
        .databunch(bs=32))

In [3]:
model = xresnet18(c_in = 3, c_out=data.c)
loss_func = nn.CrossEntropyLoss()

In [4]:
schedule = one_cycle_schedule(max_lr=5e-4)

In [5]:
cbfs = [CudaCallback, Recorder, ProgressCallback, partial(AvgStatsCallback, accuracy)]

In [6]:
learn = core.Learner(model, data, loss_func, opt_func=adam_opt(), cb_funcs=cbfs)

In [7]:
lamb_model = xresnet18(c_in = 3, c_out=data.c)

In [8]:
lamb_learner = core.Learner(lamb_model, data, loss_func, opt_func=lamb_opt(), cb_funcs=cbfs)

In [9]:
cbfs = [CudaCallback, Recorder, ProgressCallback, partial(AvgStatsCallback, accuracy)]

In [13]:
lamb_learner.fit(1, schedule)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


RuntimeError: CUDA error: unspecified launch failure

In [12]:
len(lamb_learner.opt.param_groups)

1